In [1]:
import requests
import random
import math

In [2]:
class Node:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.children = []
        self.visits = 0
        self.value = 0

    def add_child(self, child):
        self.children.append(child)

    def update(self, value):
        self.visits += 1
        self.value += value

    def ucb1(self, total_visits):
        return self.value / self.visits + math.sqrt(2 * math.log(total_visits) / self.visits)

In [3]:
class MCTS:
    def __init__(self, game, num_simulations):
        self.game = game
        self.num_simulations = num_simulations

    def search(self, root_state):
        root = Node(root_state)

        for i in range(self.num_simulations):
            print(i)
            node = self.select(root)
            result = self.simulate(node.state)
            self.backpropagate(node, result)

        return self.best_child(root)

    def select(self, node):
        while not self.game.is_terminal(node.state):
            if not self.game.is_fully_expanded(node):
                return self.expand(node)
            else:
                node =  self.best_child(node)
        return node

    def expand(self, node):
        state = self.game.random_unexpanded_child(node)
        child = Node(state, parent=node)
        node.add_child(child)
        return child

    def simulate(self, state):
        while not self.game.is_terminal(state):
            state = self.game.random_child(state)
        return self.game.reward(state)

    def backpropagate(self, node, result):
        while node is not None:
            node.update(result)
            result = -result
            node = node.parent

    def best_child(self, node):
        total_visits = sum(child.visits for child in node.children)
        return max(node.children, key=lambda child: child.ucb1(total_visits))

In [4]:
class SOR:
    def __init__(self, items, capacity, side):
        self.items = items
        self.capacity = capacity
        self.side = side

    def is_terminal(self, state):
        remaining_capacity, remaining_items, dropped_items = state
        return remaining_capacity <= 0 or len(remaining_items) == 0

    def is_fully_expanded(self, node):
        return len(node.state[1]) == len(node.children)

    def random_child(self, state):
        remaining_capacity, remaining_items, dropped_items = state
        item = random.choice(remaining_items)
        new_remaining_items = remaining_items.copy()
        new_remaining_items.remove(item)
        new_dropped_items = dropped_items.copy()
        if item[1] <= remaining_capacity:
            new_remaining_capacity = remaining_capacity - item[1]
            return new_remaining_capacity, new_remaining_items, new_dropped_items
        else:
            new_dropped_items.append(item)
            return remaining_capacity, new_remaining_items, new_dropped_items

    def random_unexpanded_child(self, node):
        remaining_capacity, remaining_items, dropped_items = node.state
        expanded_items = []
        for child in node.children:
            expanded_item = [item for item in remaining_items if item not in set(child.state[1])]
            expanded_items = list(set(expanded_items)&set(expanded_item))
        unexpanded_items = list(set(remaining_items)-set(expanded_items))
        item = random.choice(unexpanded_items)
        new_remaining_items = remaining_items.copy()
        new_remaining_items.remove(item)
        new_dropped_items = dropped_items.copy()
        if item[1] <= remaining_capacity:
            new_remaining_capacity = remaining_capacity - item[1]
            return new_remaining_capacity, new_remaining_items, new_dropped_items
        else:
            new_dropped_items.append(item)
            return remaining_capacity, new_remaining_items, new_dropped_items

    def reward(self, state):
        remaining_capacity, remaining_items, dropped_items = state
        if self.side == 'buy':
            return sum(item[0] for item in self.items) - sum(item[0] for item in remaining_items)
        else:
            return sum(item[0] for item in remaining_items) - sum(item[0] for item in self.items)
    

In [5]:
def mcts_smart_order_router(side, qty, order_type='market', price = None):
    needed_qty = qty
    route = {}
    available_qty = 0
    order = {}
    resp = requests.get("https://api.cryptowat.ch/markets/kraken/btcusd/orderbook")
    orderbook = resp.json()['result']
    ask_orderbook = orderbook['asks']
    bid_orderbook = orderbook['bids']
    if side == 'buy':
        bid_shared = []
        orderbook = ask_orderbook
        for shared in orderbook:
            if order_type == 'limit' and shared[0] <= price:
                bid_shared.append((shared[0], shared[1]))
        if len(bid_shared) == 0:
            return 'No available shared'
        sor = SOR(bid_shared, needed_qty, side)
        mcts = MCTS(sor, num_simulations=35)
        root_state = (needed_qty, bid_shared, [])
        best_state = mcts.search(root_state)
        solution = best_state
        while not len(solution.children)==0:
            solution =mcts.best_child(solution)
        packed_items = [item for item in bid_shared if ((item not in solution.state[1]) and (item not in solution.state[2]))]
        for packed_item in packed_items:
            route[packed_item[0]] = packed_item[1]
            available_qty = available_qty + packed_item[1]
        order = {'route':route, 'leave_qty': needed_qty - available_qty}
    elif side == 'sell':
        ask_shared = []
        orderbook = bid_orderbook
        for shared in orderbook:
            if order_type == 'limit' and shared[0] >= price:
                ask_shared.append((shared[0], shared[1]))
        if len(ask_shared) == 0:
            return 'No available shared'
        sor = SOR(ask_shared, needed_qty, side)
        mcts = MCTS(sor, num_simulations=35)
        root_state = (needed_qty, ask_shared, [])
        best_state = mcts.search(root_state)
        solution = best_state
        while not len(solution.children)==0:
            solution =mcts.best_child(solution)
        packed_items = [item for item in ask_shared if ((item not in solution.state[1]) and (item not in solution.state[2]))]
        for packed_item in packed_items:
            route[packed_item[0]] = packed_item[1]
            available_qty = available_qty + packed_item[1]
        order = {'route':route, 'leave_qty': needed_qty - available_qty}
    return order

In [6]:
def greedy_smart_order_router(side, qty, order_type='market', price=None):
    needed_qty = qty
    route = {}
    available_qty = 0
    resp = requests.get("https://api.cryptowat.ch/markets/kraken/btcusd/orderbook")
    orderbook = resp.json()['result']
    ask_orderbook = orderbook['asks']
    bid_orderbook = orderbook['bids']
    if side == 'buy':
        orderbook = ask_orderbook
        if orderbook is None or len(orderbook) == 0:
            return 'No available shared'
        ask_shared = sorted(orderbook, key=lambda x:x[0], reverse=False)
        for shared in ask_shared:
            if qty <= 0:
                break
            ask_price = shared[0]
            ask_qty = shared[1]
            if order_type == 'market' or ask_price <= price:
                if ask_qty >= qty and qty>0:
                    route[ask_price] = qty
                    available_qty += qty
                    qty = 0
                elif qty > 0:
                    route[ask_price] = ask_qty
                    available_qty += ask_qty
                    qty = qty - ask_qty
    elif side == 'sell':
        orderbook = bid_orderbook
        if orderbook is None or len(orderbook) == 0:
            return 'No available shared'
        bid_shared = sorted(orderbook, key=lambda x:x[0], reverse=True)
        for shared in bid_shared:
            if qty <= 0:
                break
            bid_price = shared[0]
            bid_qty = shared[1]
            if order_type == 'market' or bid_price >= price:
                if bid_qty >= qty and qty>0:
                    route[bid_price] = qty
                    available_qty += qty
                    qty = 0
                elif qty > 0:
                    route[bid_price] = bid_qty
                    available_qty += bid_qty
                    qty = qty - bid_qty
    order = {'route':route, 'leave_qty':needed_qty - available_qty}
    return order    

In [10]:
side = 'buy'
qty = 1
price = 30000
greedy_smart_order_router(side, qty, 'limit', price)

{'route': {27581.5: 0.40930392,
  27582.8: 0.001,
  27583.4: 0.00114731,
  27583.5: 0.58854877},
 'leave_qty': 0.0}